# Word Alignment Tutorial

In this notebook, we will demonstrate how to use machine to train statistical word alignment models and then use them to predict alignments between sentences. Machine uses the [Thot](https://github.com/sillsdev/thot) library to implement word alignment models. The implementations are contained in the [SIL.Machine.Translation.Thot](https://www.nuget.org/packages/SIL.Machine.Translation.Thot/) package.


In [1]:
#r "nuget:SIL.Scripture,12.0.1"
#r "nuget:Thot"
#r "../src/SIL.Machine.Translation.Thot/bin/Debug/netstandard2.0/SIL.Machine.dll"
#r "../src/SIL.Machine.Translation.Thot/bin/Debug/netstandard2.0/SIL.Machine.Translation.Thot.dll"

void WriteLine(string text = "")
{
    Console.Write(text + "\n");
}

Installed Packages SIL.Scripture, 12.0.1 Thot, 3.4.1

## Training models

The first step in training a statistical word alignment model is setting up a parallel text corpus. Word alignment models are unsupervised, so they only require a parallel text corpus to train. Manually created alignments are not necessary. So let's create a parallel corpus from the source and target monolingual corpora.


In [2]:
using SIL.Machine.Corpora;
using SIL.Machine.Tokenization;

var sourceCorpus = new ParatextTextCorpus("data/VBL-PT");
var targetCorpus = new ParatextTextCorpus("data/WEB-PT");
var parallelCorpus = sourceCorpus.AlignRows(targetCorpus).Tokenize<LatinWordTokenizer>();

Machine has implementations of all common statistical models, including the famous IBM models (1-4), HMM, and FastAlign. All alignment models implement the `IWordAlignmentModel` interface. This makes it easier to swap out different models in your code.


In this tutorial, we are going to start by training an IBM-1 model. There are two possible ways to train a model. First, we will demonstrate training a model from a class that implements `IWordAlignmentModel`. We use the `CreateTrainer` method to create a trainer object that is used to train the model. If we do not specify a file path when creating the model object, then the model will only exist in memory. When we call the `SaveAsync` method, the model instance is updated with the trained model parameters, but the model is not written to disk. We are going to use the `Lowercase` token processor to pre-process that data, since that generally gives better results.


In [3]:
using SIL.Machine.Translation.Thot;
using SIL.Machine.Utils;

{
    using var model = new ThotIbm1WordAlignmentModel();
    using var trainer = model.CreateTrainer(parallelCorpus.Lowercase());
    await trainer.TrainAsync(new DelegateProgress(status => WriteLine($"Training IBM-1 model: {status.PercentCompleted:P}")));
    await trainer.SaveAsync();
}

Training IBM-1 model: 0.00%
Training IBM-1 model: 16.67%
Training IBM-1 model: 33.33%
Training IBM-1 model: 50.00%
Training IBM-1 model: 66.67%
Training IBM-1 model: 83.33%
Training IBM-1 model: 100.00%


The other option for training a model is to construct a trainer object directly. This method is useful for when you are only interested in training the model and saving it to disk for later use. We need to specify where the model will be saved after it is trained and we call the `SaveAsync` method.


In [4]:
using System.IO;

Directory.CreateDirectory("out/VBL-WEB-IBM1");
{
    using var trainer = new ThotWordAlignmentModelTrainer(ThotWordAlignmentModelType.Ibm1, parallelCorpus.Lowercase(),
        "out/VBL-WEB-IBM1/src_trg");
    await trainer.TrainAsync(new DelegateProgress(status => WriteLine($"Training IBM-1 model: {status.PercentCompleted:P}")));
    await trainer.SaveAsync();
    WriteLine("IBM-1 model saved");
}

Training IBM-1 model: 0.00%
Training IBM-1 model: 16.67%
Training IBM-1 model: 33.33%
Training IBM-1 model: 50.00%
Training IBM-1 model: 66.67%
Training IBM-1 model: 83.33%
Training IBM-1 model: 100.00%
IBM-1 model saved


## Aligning parallel sentences

Now that we have a trained alignment model, we can find the best alignment for a parallel sentence. We call `Align` method to find the best alignment. The results are returned as a `WordAlignmentMatrix` object.


In [5]:
{
    using var model = new ThotIbm1WordAlignmentModel("out/VBL-WEB-IBM1/src_trg");
    foreach (var row in parallelCorpus.Take(5))
    {
        var alignment = model.Align(row.SourceSegment.Lowercase(), row.TargetSegment.Lowercase());

        WriteLine($"{row.Ref}");
        WriteLine($"Source: {row.SourceText}");
        WriteLine($"Target: {row.TargetText}");
        WriteLine($"Alignment: {alignment}");
    }
}

1JN 1:1
Source: Esta carta trata sobre la Palabra de vida que existía desde el principio , que hemos escuchado , que hemos visto con nuestros propios ojos y le hemos contemplado , y que hemos tocado con nuestras manos .
Target: That which was from the beginning , that which we have heard , that which we have seen with our eyes , that which we saw , and our hands touched , concerning the Word of life
Alignment: 2-2 2-20 2-25 2-29 2-30 3-32 4-4 4-33 5-34 6-35 7-36 12-3 12-5 13-6 13-12 13-21 13-26 13-31 15-0 15-7 15-10 15-13 15-16 15-22 16-1 16-8 16-11 16-14 16-23 20-17 21-18 22-19 22-28 25-27 35-9 35-15 35-24
1JN 1:2
Source: Esta Vida nos fue revelada . La vimos y damos testimonio de ella . Estamos hablándoles de Aquél que es la Vida Eterna , que estaba con el Padre , y que nos fue revelado .
Target: ( and the life was revealed , and we have seen , and testify , and declare to you the life , the eternal life , which was with the Father , and was revealed to us ) ;
Alignment: 0-17 0-35 1-

## Getting model probabilities

A statistical word alignment model consists of one or more conditional probability distributions that are estimated from the training data. For example, most models estimate a word translation probability distribution that can be queried to obtain the probability that a source word is a translation of a target word. Each model class has methods to obtain these probabilities. Let's try getting some translation probabilities from the IBM-1 model that we trained by calling the `GetTranslationProbability` method. In order to get the probability that a word does not translate to anything, you can pass `null` instead of the word string.


In [6]:
{
    using var model = new ThotIbm1WordAlignmentModel("out/VBL-WEB-IBM1/src_trg");
    double prob = model.GetTranslationProbability("es", "is");
    WriteLine($"es -> is: {prob:0.0000}");
    prob = model.GetTranslationProbability(null, "that");
    WriteLine($"NULL -> that: {prob:0.0000}");
}

es -> is: 0.2720
NULL -> that: 0.0516


## Symmetrized alignment models

Most statistical word alignment models are directional and asymmetric. This means that it can only model one-to-one and one-to-many alignments in one direction. They are not capable of modeling many-to-many alignments, which can occur in some language pairs. One way to get around this limitation is to train models in both directions (source-to-target and target-to-source), and then merge the resulting alignments from the two models into a single alignment. This is called symmetrization and is a common practice when using statistical word alignment models. In addition, researchers have found that symmetrized alignments are better quality.


Machine provides a special word alignment model class to support symmetrization called `SymmetrizedWordAlignmentModel`. Let's demonstrate how to use this class. First, we will train the symmetrized model using the `SymmetrizedWordAlignmentModelTrainer` class.


In [7]:
using SIL.Machine.Translation;

{
    using var srcTrgTrainer = new ThotWordAlignmentModelTrainer(ThotWordAlignmentModelType.Ibm1,
        parallelCorpus.Lowercase(), "out/VBL-WEB-IBM1/src_trg");
    using var trgSrcTrainer = new ThotWordAlignmentModelTrainer(ThotWordAlignmentModelType.Ibm1,
        parallelCorpus.Invert().Lowercase(), "out/VBL-WEB-IBM1/trg_src");
    using var symmetrizedTrainer = new SymmetrizedWordAlignmentModelTrainer(srcTrgTrainer, trgSrcTrainer);
    await symmetrizedTrainer.TrainAsync(new DelegateProgress(status =>
        WriteLine($"{status.Message}: {status.PercentCompleted:P}")));
    await symmetrizedTrainer.SaveAsync();
    WriteLine("Symmetrized IBM-1 model saved");
}

Training direct alignment model: 0.00%
Training direct alignment model: 0.00%
Training direct alignment model: 8.33%
Training direct alignment model: 16.67%
Training direct alignment model: 25.00%
Training direct alignment model: 33.33%
Training direct alignment model: 41.67%
Training direct alignment model: 50.00%
Training inverse alignment model: 50.00%
Training inverse alignment model: 50.00%
Training inverse alignment model: 58.33%
Training inverse alignment model: 66.67%
Training inverse alignment model: 75.00%
Training inverse alignment model: 83.33%
Training inverse alignment model: 91.67%
Training inverse alignment model: 100.00%
Symmetrized IBM-1 model saved


The model can also be trained using the `CreateTrainer` method on `SymmetrizedWordAlignmentModel`. Now that we've trained the symmetrized model, let's obtain some alignments. Machine supports many different symmetrization heuristics. The symmetrization heuristic to use when merging alignments can be specified using the `Heuristic` property. In this case, we will use the `GrowDiagFinalAnd` heuristic.


In [8]:
{
    using var srcTrgModel = new ThotIbm1WordAlignmentModel("out/VBL-WEB-IBM1/src_trg");
    using var trgSrcModel = new ThotIbm1WordAlignmentModel("out/VBL-WEB-IBM1/trg_src");
    using var symmetrizedModel = new SymmetrizedWordAlignmentModel(srcTrgModel, trgSrcModel)
    {
        Heuristic = SymmetrizationHeuristic.GrowDiagFinalAnd
    };
    foreach (var row in parallelCorpus.Take(5))
    {
        var alignment = symmetrizedModel.Align(row.SourceSegment.Lowercase(), row.TargetSegment.Lowercase());

        WriteLine($"{row.Ref}");
        WriteLine($"Source: {row.SourceText}");
        WriteLine($"Target: {row.TargetText}");
        WriteLine($"Alignment: {alignment}");
    }
}

1JN 1:1
Source: Esta carta trata sobre la Palabra de vida que existía desde el principio , que hemos escuchado , que hemos visto con nuestros propios ojos y le hemos contemplado , y que hemos tocado con nuestras manos .
Target: That which was from the beginning , that which we have heard , that which we have seen with our eyes , that which we saw , and our hands touched , concerning the Word of life
Alignment: 1-25 2-25 3-32 4-4 4-33 5-34 6-35 7-36 8-0 12-5 13-6 15-10 16-11 19-20 20-17 21-18 22-19 25-27 35-9
1JN 1:2
Source: Esta Vida nos fue revelada . La vimos y damos testimonio de ella . Estamos hablándoles de Aquél que es la Vida Eterna , que estaba con el Padre , y que nos fue revelado .
Target: ( and the life was revealed , and we have seen , and testify , and declare to you the life , the eternal life , which was with the Father , and was revealed to us ) ;
Alignment: 0-17 1-3 2-36 3-4 4-0 6-38 7-0 8-1 9-0 10-0 11-13 12-18 14-8 18-9 22-23 23-6 26-28 27-2 28-29 28-30 33-4 34-5
1JN